In [ ]:
import gradio as gr

with gr.Blocks() as demo:

    def click_send(prompt):
        print(prompt)
        return prompt
    
    gr.Markdown("# 나의 GPT 월드")
    chatbot = gr.Chatbot(label="나의 GPT", height=150)
    
    with gr.Row():
        prompt_textbox = gr.Textbox(label="프롬프트", placeholder="프롬프트를 입력하세요.", scale=6)
        send_button = gr.Button("전송", scale=1)
    
    return_textbox = gr.Textbox(label="return")
    send_button.click(click_send, inputs=[prompt_textbox], outputs=[return_textbox])

demo.launch()



In [ ]:
def click_send(prompt, histories):
    print(prompt)
    histories.append(("안녕?", "반가워!!"))

    print(histories)
    return histories

with gr.Blocks() as demo:
    
    gr.Markdown("# 나의 GPT 월드")
    chatbot = gr.Chatbot(label="나의 GPT", height=150)
    
    with gr.Row():
        prompt_textbox = gr.Textbox(label="프롬프트", placeholder="프롬프트를 입력하세요.", scale=6)
        send_button = gr.Button("전송", scale=1)
    
    send_button.click(click_send, inputs=[prompt_textbox, chatbot], outputs=[chatbot])

demo.launch()

In [ ]:
import requests  # HTTP 요청을 보내기 위한 requests 라이브러리 불러오기
import re
import os
from dotenv import load_dotenv
import gradio as gr


# .env 파일 로드
load_dotenv()

# 환경 변수 가져오기
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
ai_search_endpoint = os.getenv("AI_SEARCH_ENDPOINT")
ai_search_api_key = os.getenv("AI_SEARCH_API_KEY")
ai_search_index_name = os.getenv("AI_SEARCH_INDEX")
ai_search_semantic = os.getenv("AI_SEARCH_SEMANTIC")
system_content = os.getenv("SYSTEM_CONTENT")

def request_gpt(prompt):
    # 요청 헤더 설정
    headers = {
        "Content-Type": "application/json",  # 요청 본문을 JSON 형식으로 지정
        "api-key": api_key  # 인증을 위한 API 키 포함
    }

    # 요청 본문 (채팅 메시지 설정)
    body = {
        "messages": [
            {
                "role": "system",
                "content": system_content
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        "temperature": 0.7,  # 창의성 조절 (0: 보수적, 1: 창의적)
        "top_p": 0.9,  # 확률 분포 기반 단어 선택 (높을수록 다양한 단어 선택 가능)
        "max_tokens": 4096,
        "stop":None,  
        "stream":False,
        "data_sources": [
            {
            "type": "azure_search",
            "parameters": {
                "endpoint": ai_search_endpoint,
                "index_name": ai_search_index_name,
                "semantic_configuration": ai_search_semantic,
                "query_type": "semantic",
                "fields_mapping": {},
                "in_scope": True,
                "filter": None,
                "strictness": 2,
                "top_n_documents": 8,
                "authentication": {
                "type": "api_key",
                "key": ai_search_api_key
                },
                "key": ai_search_api_key,
            }
            }
        ],
    }


    # OpenAI API에 POST 요청을 보내고 응답을 받아옴
    response = requests.post(endpoint, headers=headers, json=body)
    if response.status_code == 200:
        response_json = response.json()  # 응답을 JSON 형식으로 변환

        # AI의 응답에서 채팅 메시지 추출
        message = response_json['choices'][0]['message']
        citation_list = message['context']['citations']

        # 부적절한 콘텐츠 관리 필터에 걸린 게 있는지 그 결과를 출력하는 것. 수업에서는 활용 안함.
        # content_filter = response_json['choices'][0]['content_filter_results']
        # 역할 (예: assistant)
        role = message['role']
        # 응답 메시지 (AI의 답변)
        content = message['content']
        content = re.sub(r'\[doc(\d+)\]', r'[참조 \1]', content)
        # 역할 출력 (예: assistant)
        # print(role)
        return content, citation_list
    else:
        return "", list()


################################ GRADIO 부분

with gr.Blocks() as demo:
    
    def click_send(prompt, histories):
        print(prompt)
        content, citation_list = request_gpt(prompt=prompt)
        message_list = [{"role": "user", "content": prompt}, {"role": "assistant", "content": content}]
        histories.extend(message_list)
        # histories.append((prompt, content))
        # histories.append(gr.ChatMessage(role="user", content="안녕?"))
        # histories.append(gr.ChatMessage(role="assistant", content="반가워!!!?"))

        citation_list_format = ""

        for index, c in enumerate(citation_list, start=1):
            lines = c['content'].splitlines()
            citation_list_format += f"참조 {index} : {lines[-1]} {lines[-2]}번 {lines[0]} {lines[1]}\n\n"

        print(histories)
        return histories, citation_list_format.strip(), ""
    
    gr.Markdown("# 나의 GPT 월드")
    
    with gr.Row():
      chatbot = gr.Chatbot(label="나의 GPT", type="messages")
      citation_textbox = gr.Textbox(label="참조")
    
    with gr.Row():
        prompt_textbox = gr.Textbox(label="프롬프트", placeholder="프롬프트를 입력하세요.", scale=6)
        send_button = gr.Button("전송", scale=1)
        
    send_button.click(click_send, inputs=[prompt_textbox, chatbot], outputs=[chatbot, citation_textbox, prompt_textbox])
    
    
demo.launch()

